In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import scipy 
import sklearn
from collections import Counter
from sklearn.metrics import multilabel_confusion_matrix
from scipy import spatial

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
data = pd.read_csv('../input/kmeans-data/data.csv')
labels = pd.read_csv('../input/kmeans-data/label.csv',names=['label'],header=None)

In [3]:
data.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data.count()

0        9999
0.1      9999
0.2      9999
0.3      9999
0.4      9999
         ... 
0.663    9999
0.664    9999
0.665    9999
0.666    9999
0.667    9999
Length: 784, dtype: int64

In [5]:
#Split data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split( data, test_size=0.08, random_state=42)
train_labels, test_labels = train_test_split( labels, test_size=0.08, random_state=42)

In [6]:
class KMeans:
    
    def calculate_SSE(self, centroid_value_dict, centroid_dict,data):
        sse_data = 0
        for i in centroid_dict:
            sse_cluster = 0

            for j in centroid_dict[i]:
                dp = list(data.iloc[int(j)])
                for a,b in zip(centroid_value_dict[i],dp):
                    sse_cluster += (a-b)**2
            sse_data+=sse_cluster
        return sse_data    
    
    def Initialize_Centroids(self,data,K):
        m = data.shape[0]
        centroid_value_dict={}
        for i in range(K):
            r = np.random.randint(0, m-1)
            centroid_value_dict[i] = data.iloc[r]
        return centroid_value_dict
    
    def jaccard_similarity(self,centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

    def train_Kmeans(self,data,K,max_iter=20,mode=1,tol=10):
        #Mode = 1 => Euclidean np.linalg.norm(x-list(data.iloc[i,:]))
        #Mode = 2 => Jaccard
        #Mode = 3 => Cosine
        centroid_value_dict = self.Initialize_Centroids(data,K)
        new_centroid_value_dict = {}
        count = 0
        centroid_dict = {}
        convergence = False
        while((count<max_iter) and not convergence):
            
            for i in list(centroid_value_dict.keys()):
                centroid_dict[i]=[]
            for i in range(data.shape[0]):
                x = data.iloc[i]
                if mode==1 :
                    distance_measure = [np.linalg.norm(x-centroid_value_dict[j])  for j in centroid_value_dict]
                    idx = np.argmin(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==2 :
                    distance_measure = [self.jaccard_similarity(list(x),centroid_value_dict[j]) for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==3 :
                    distance_measure = [1-scipy.spatial.distance.cosine(x,list(centroid_value_dict[j]))  for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                
                prev_centroids=dict(centroid_value_dict)
                
            
            for i in centroid_dict:
                if len(centroid_dict[i]):
                    dps_centroid = centroid_dict[i]
                    centroid_value_dict[i] = np.average(data.iloc[dps_centroid],axis=0)
            
            
            current_tol=-1
            for i in centroid_value_dict:
                prev_centroid_point = prev_centroids[i]
                new_centroid_point = centroid_value_dict[i]
                change = np.sum(np.absolute(new_centroid_point-prev_centroid_point))
                current_tol = max(change, current_tol)
                
            print("Tolerance for the Iteration ",count,": ",current_tol)
            
            count+=1
            if (current_tol<10):
                convergence = True
                break
           # print("KMeans Iteration",count)
        return centroid_value_dict,centroid_dict
    

In [7]:
def predict_cluster_labels(C, S, labels):
    '''
    Input : C -> Centroids
            S -> Set of Indicies corresponding to Centroid C
            data -> Data used to form clusters
    Output : Returns an array of size K having labels based on majority voting in the cluster
    '''
    cluster_labels = np.zeros(10,dtype=int)
    for c in C:
        labels_of_points = []
        for point in S[c]:
            labels_of_points.extend(labels.iloc[point])
        counter = Counter(labels_of_points)
        try:
            cluster_labels[c] = max(counter, key=counter.get)
        except:
            cluster_labels[c] = np.random.randint(0,9)
    return cluster_labels

In [8]:
def jaccard_similarity(centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

In [9]:
def accuracy(centroids, centroid_Labels, test_data, true_labels, mode=1):
    y_true = list(true_labels['label']);
    y_pred = []
    for index in range(test_data.shape[0]):
        featureset = test_data.iloc[index]
        if mode==1:
            distances = [np.linalg.norm(featureset - centroids[centroid]) for centroid in centroids]
            classification = distances.index(min(distances))
            y_pred.append(centroid_Labels[classification])
        elif mode==2:
            similarity = [jaccard_similarity(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification]) 
        elif mode==3:
            similarity = [1 - spatial.distance.cosine(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification])
    denominator = test_data.shape[0]
    correctly_classified = 0
    for i in range(0,len(y_pred)):
        if y_true[i] == y_pred[i]:
            correctly_classified += 1
    accuracy = correctly_classified/denominator
    return accuracy

In [10]:
model1 = KMeans()
centroids1,clusters1 = model1.train_Kmeans(data,10, max_iter=100,mode=1)

Tolerance for the Iteration  0 :  28253.305982905986
Tolerance for the Iteration  1 :  5203.900750848727
Tolerance for the Iteration  2 :  4138.2160355396645
Tolerance for the Iteration  3 :  3822.7612913918692
Tolerance for the Iteration  4 :  3698.3624468072576
Tolerance for the Iteration  5 :  1835.4912117902782
Tolerance for the Iteration  6 :  798.475158869518
Tolerance for the Iteration  7 :  588.550544411313
Tolerance for the Iteration  8 :  411.74694522180033
Tolerance for the Iteration  9 :  234.72926995485136
Tolerance for the Iteration  10 :  238.8342935487566
Tolerance for the Iteration  11 :  286.3391003675474
Tolerance for the Iteration  12 :  254.57018669646163
Tolerance for the Iteration  13 :  186.4844131176693
Tolerance for the Iteration  14 :  187.03352286126704
Tolerance for the Iteration  15 :  173.98646601406895
Tolerance for the Iteration  16 :  140.23445535675714
Tolerance for the Iteration  17 :  160.02006146020386
Tolerance for the Iteration  18 :  131.0088080

In [11]:
Euclidean_SSE = model1.calculate_SSE(centroids1,clusters1,data)

In [12]:
print("Euclidean SSE:",Euclidean_SSE)

Euclidean SSE: 25316242143.801693


In [13]:
cluster_labels1 = predict_cluster_labels(centroids1,clusters1,labels)
cluster_labels1

array([5, 0, 3, 2, 1, 8, 7, 3, 0, 9])

In [14]:
Accuracy_Euclidean = accuracy(centroids1, cluster_labels1,test_data,test_labels)
Accuracy_Euclidean

0.0875

In [15]:
model2 = KMeans()
centroids2,clusters2 = model2.train_Kmeans(data,10, max_iter=100,mode=2)
Jaccard_SSE = model2.calculate_SSE(centroids2,clusters2,data)

Tolerance for the Iteration  0 :  34565.31001890359
Tolerance for the Iteration  1 :  17900.874105197636
Tolerance for the Iteration  2 :  9955.762886597939
Tolerance for the Iteration  3 :  16293.732413241325
Tolerance for the Iteration  4 :  3794.053178389067
Tolerance for the Iteration  5 :  722.669134882238
Tolerance for the Iteration  6 :  570.85104667672
Tolerance for the Iteration  7 :  589.711590093216
Tolerance for the Iteration  8 :  1685.0883827513185
Tolerance for the Iteration  9 :  952.5932035982056
Tolerance for the Iteration  10 :  961.2304131320847
Tolerance for the Iteration  11 :  227.70706020181848
Tolerance for the Iteration  12 :  631.8978648237774
Tolerance for the Iteration  13 :  372.7988996060237
Tolerance for the Iteration  14 :  0.0


In [16]:
print("Jacard SSE:",Jaccard_SSE)

Jacard SSE: 34361687572.938736


In [17]:
cluster_labels2 = predict_cluster_labels(centroids2,clusters2,labels)
cluster_labels2

array([1, 2, 2, 7, 6, 4, 2, 0, 1, 8])

In [18]:
Accuracy_Jaccard = accuracy(centroids2, cluster_labels2,test_data,test_labels)
Accuracy_Jaccard

0.1075

In [19]:
model3 = KMeans()
centroids3,clusters3 = model3.train_Kmeans(data,10, max_iter = 100,mode=3)

Tolerance for the Iteration  0 :  34077.41536539662
Tolerance for the Iteration  1 :  3521.3462677165357
Tolerance for the Iteration  2 :  1938.1517337880973
Tolerance for the Iteration  3 :  2652.0924288362794
Tolerance for the Iteration  4 :  4663.580094086264
Tolerance for the Iteration  5 :  3493.2876479191596
Tolerance for the Iteration  6 :  1951.650176409922
Tolerance for the Iteration  7 :  1720.9151701791875
Tolerance for the Iteration  8 :  1329.2509867707656
Tolerance for the Iteration  9 :  1066.5759177770528
Tolerance for the Iteration  10 :  1385.8849019832626
Tolerance for the Iteration  11 :  1111.922217073024
Tolerance for the Iteration  12 :  873.3226854861178
Tolerance for the Iteration  13 :  1061.7961146337232
Tolerance for the Iteration  14 :  928.4854553035341
Tolerance for the Iteration  15 :  783.2714551444342
Tolerance for the Iteration  16 :  710.6197711614174
Tolerance for the Iteration  17 :  675.4640857924
Tolerance for the Iteration  18 :  676.03721931800

In [20]:
Cosine_SSE = model3.calculate_SSE(centroids3,clusters3,data)

In [21]:
print("Euclidean SSE:",Euclidean_SSE)
print("Jacard SSE:",Jaccard_SSE)
print("Cosine SSE :",Cosine_SSE)

Euclidean SSE: 25316242143.801693
Jacard SSE: 34361687572.938736
Cosine SSE : 25416927048.0813


In [22]:
cluster_labels3 = predict_cluster_labels(centroids3,clusters3,labels)
cluster_labels3

array([1, 9, 5, 3, 2, 0, 3, 0, 8, 7])

In [23]:
Accuracy_Cosine = accuracy(centroids3, cluster_labels3,test_data,test_labels)
Accuracy_Euclidean
Accuracy_Jaccard
Accuracy_Cosine

0.0775

In [24]:
print("Euclidean accuracy:",Accuracy_Euclidean)
print("Jacard accuracy:",Accuracy_Jaccard)
print("Cosine accuracy :",Accuracy_Cosine)

Euclidean accuracy: 0.0875
Jacard accuracy: 0.1075
Cosine accuracy : 0.0775
